## ML Model to predict best move

Much of code is taken from the William Lifferth's Kaggle Notebook. (Thank You!)  
Resulted in 0.12 score on the public and 0.15 on private leaderboard.

In [ ]:
import pandas as pd
import numpy as np
import chess

In [ ]:
train_df = pd.read_csv('/kaggle/input/train-an-ai-to-play-chess/train.csv', index_col='id')

In [ ]:
train_df.head()

In [ ]:
train_df = train_df[:55000]
val_df = train_df[:-1000]

### Preprocessing the FEN notation

In [ ]:
def one_hot_encode_peice(piece):
    pieces = list('rnbqkpRNBQKP.')
    arr = np.zeros(len(pieces))
    piece_to_index = {p: i for i, p in enumerate(pieces)}
    index = piece_to_index[piece]
    arr[index] = 1
    return arr

### Custom Encoding for pieces

Rather than using the one hot encoding for pieces, I assigned these custom values to them.
Although one hot encoding is recommended, this custom encoding provided slightly better results on leaderboard 😅.  
Used positive piece value for White Pieces and negative for Black Pieces.  
I feel Bishop == 3.5 rather 3.

In [ ]:
def encode_board(board):
    # first lets turn the board into a string
    board_str = str(board)
    # then lets remove all the spaces
    material_dict = {
        'p': -1,
        'b': -3.5,
        'n': -3,
        'r': -5,
        'q': -9,
        'k': -4,
        'K': 4,
        '.': 0,
        'P': 1,
        'B': 3.5,
        'N': 3,
        'R': 5,
        'Q': 9,
    }
    board_str = board_str.replace(' ', '')
    board_list = []
    for row in board_str.split('\n'):
        row_list = []
        for piece in row:
            # print(piece)
            row_list.append(material_dict.get(piece))
        board_list.append(row_list)
    return np.array(board_list)

In [ ]:
encode_board(chess.Board())

In [ ]:
def encode_fen_string(fen_str):
    board = chess.Board(fen=fen_str)
    return encode_board(board)

In [ ]:
X_train = np.stack(train_df['board'].apply(encode_fen_string))
y_train = train_df['black_score']

In [ ]:
X_val = np.stack(val_df['board'].apply(encode_fen_string))
y_val = val_df['black_score']

### Model and Training

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout

# With the Keras Sequential model we can stack neural network layers together
model = Sequential([
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(1),
])

model.compile(
    optimizer='adam',
    loss='mean_squared_error')

In [ ]:
# To test things out, let's train for 20 epochs and see how our model is doing
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_val, y_val))

In [ ]:
model.save('./model_512_128_1_other')

In [ ]:
import matplotlib.pyplot as plt

# Lets plot the history of our training session to see how things progressed over time
plt.style.use('ggplot')
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.title('Loss During Training')
plt.show()

In [ ]:
def play_nn(fen, player='b'):
    # We can create a python-chess board instance from the FEN string like this:
    board = chess.Board(fen=fen)

    # And then evaluate all legal moves
    best_move = ''
    worst_move = ''
    minScore = float('inf')
    maxScore = float('-inf')

    moves = []
    for move in board.legal_moves:
        # For each move, we'll make a copy of the board and try that move out
        candidate_board = board.copy()
        candidate_board.push(move)
        input_vector = encode_board(str(candidate_board)).astype(np.int32).flatten()
        
        # This is where our model gets to shine! It tells us how good the resultant score board is for black:
        score = model.predict(np.expand_dims(input_vector, axis=0), verbose=0)[0][0]
        if score > maxScore:
            best_move = move
            maxScore = score
        elif score < minScore:
            worst_move = move
            minScore = score
        # moves.append((score, move))
        # if show_move_evaluations:
        #     print(f'{move}: {score}')
    
    # By default sorting our moves will put the lowest scores at the top.
    # This would give us the right answer if we were playing as white,
    # but if we're playing as black we want to reverse things (then grab the first move):
    
    if(player=='b'):
        return str(worst_move)
    # Now we turn our move into a string, return it and call it a day!
    return str(best_move)

In [ ]:
from IPython.display import SVG, display

# Our play function accepts whatever strategy our AI is using, like play_random from above
def play_game(ai_function):
    board = chess.Board()

    while board.outcome() is None:
        # We print out the board as an SVG
        display(SVG(board._repr_svg_()))

        # If it's white's turn, we have the user play
        if board.turn == chess.WHITE:
            user_move = input('Your move: ')
            if user_move == 'quit':
                break
            # The move a user puts in isn't a valid move, we keep prompting them for a valid move
            while user_move not in [str(move) for move in board.legal_moves]:
                print('That wasn\'t a valid move. Please enter a move in Standard Algebraic Notation')
                user_move = input('Your move: ')
            board.push_san(user_move)

        # If it's black's turn, we have the AI play
        elif board.turn == chess.BLACK:
            ai_move = ai_function(board.fen())
            print(f'AI move: {ai_move}')
            board.push_san(ai_move)
    print(board.outcome())


### Inference

In [ ]:
test_df = pd.read_csv('/kaggle/input/train-an-ai-to-play-chess/test.csv')

In [ ]:
test_df.head()

In [ ]:
test_df['best_move'] = test_df['board'].apply(play_nn)

In [ ]:
test_df.head()

In [ ]:
submission = pd.read_csv('/kaggle/input/train-an-ai-to-play-chess/sample_submission.csv')
submission['best_move'] = test_df['best_move']

In [ ]:
submission.to_csv('nn_sub_4.csv', index=False)

In [ ]:
nan